In [14]:
#This script runs a transient simulation of the CO2 methanation in a Berty (CSTR) reactor.
#For details on the setup and the model refer to the following publication 
#https://doi.org/10.1002/cctc.202200570

#The feed is alternated between a mixture of H2/Ar and CO2/He

import cantera as ct
import numpy as np
import pandas as pd

#Parameters for the Berty reactor
vol=98e-6
cat_area_per_vol = 1.1198/vol  # Catalyst particle surface area 
cat_area = cat_area_per_vol * vol
sccm = 395*1e-6/60

#Flow rates
V_purge=95

#Calculate the H2/Ar stream
V_Ar=210
V_H2=90

V1_ges=V_purge+V_H2+V_Ar
x_Ar_1=(V_Ar+V_purge)/V1_ges
x_H2_1=V_H2/V1_ges

#Calculate the H2/Ar stream
V_He=210
V_CO2=90
V2_ges=V_purge+V_He+V_CO2
x_Ar_2=V_purge/V2_ges
x_He_2=V_He/V2_ges
x_CO2_2=V_CO2/V2_ges

n_runs=5000
offset=0

def methanation(t,p):
        T=t+273.15 #temperature
        pressure=p*1e5 #pressure of the CSTR

        file='mechanism.yaml'
        cti_file = file

        #The CSTR setup in Cantera is quite straightforward
        gas = ct.Solution(cti_file, 'gas')
        gas.TPX = T, pressure, 'Ar:0.7722, He:0, Ne:0.0, CO2(2):0,  H2(4):0.2278'
        # import the surface model
        surf = ct.Interface(cti_file,'surface1', [gas])
        surf.TP = T, pressure
        surf.coverages = {'site(7)':1}

        upstream = ct.Reservoir(gas, name='upstream')
        downstream = ct.Reservoir(gas, name='downstream')
        r=ct.IdealGasReactor(gas, energy='off')

        r.volume = vol
        rsurf=ct.ReactorSurface(surf, r, A=cat_area)

        mass_flow=sccm* gas.mean_molecular_weight*1e5/ct.gas_constant/273.15

        mflow = ct.MassFlowController(upstream, r, mdot=mass_flow)
        v = ct.PressureController(r, downstream, master=mflow, K=1e-9)
        sim=ct.ReactorNet([r])

        ##set relative and absolute tolerances on the simulation
        sim.rtol = 1.0e-12
        sim.atol = 1.0e-26
        sim.max_err_test_fails = 100
        time_lag=16 #empirical parameter to align the internal standards. Lag is caused
# by switching time, residence time in the pipes and stuff

        rxn_time=np.linspace(0,1050+time_lag,1051+time_lag)

        gas_mole_fracs=np.zeros([gas.n_species, len(rxn_time)])
        surf_fracs=np.zeros([surf.n_species, len(rxn_time)])

        for i in range(len(rxn_time)):
            time=rxn_time[i]
            #Switch manually the inlet concentration between both feeds
            if time < 180+time_lag:
                upstream.thermo.TPX=T,pressure, 'Ar:0.7722, He:0, Ne:0.0, CO2(2):0,  H2(4):0.2278'
            elif (time >= 180+time_lag) and (time < 240+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.2405, He:0.5316, Ne:0.0, CO2(2):0.2278,  H2(4):0'
            elif (time >= 240+time_lag) and (time < 300+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.7722, He:0, Ne:0.0, CO2(2):0,  H2(4):0.2278'
            elif (time >= 300+time_lag) and (time < 360+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.2405, He:0.5316, Ne:0.0, CO2(2):0.2278,  H2(4):0'
            elif (time >= 360+time_lag) and (time < 420+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.7722, He:0, Ne:0.0, CO2(2):0,  H2(4):0.2278'
            elif (time >= 420+time_lag) and (time < 480+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.2405, He:0.5316, Ne:0.0, CO2(2):0.2278,  H2(4):0'
            elif (time >= 480+time_lag) and (time < 540+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.7722, He:0, Ne:0.0, CO2(2):0,  H2(4):0.2278'
            elif (time >= 540+time_lag) and (time < 600+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.2405, He:0.5316, Ne:0.0, CO2(2):0.2278,  H2(4):0'
            elif (time >= 600+time_lag) and (time < 660+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.7722, He:0, Ne:0.0, CO2(2):0,  H2(4):0.2278'
            elif (time >= 660+time_lag) and (time < 720+time_lag):
                upstream.thermo.TPX=T,pressure, 'Ar:0.2405, He:0.5316, Ne:0.0, CO2(2):0.2278,  H2(4):0'
            else:
                upstream.thermo.TPX=T,pressure, 'Ar:0.7722, He:0, Ne:0.0, CO2(2):0,  H2(4):0.2278'

            upstream.syncState()
            #The volumetric flow rate is the same but the mass flow changes
            mass_flow=sccm* upstream.thermo.mean_molecular_weight*1e5/ct.gas_constant/273.15
            mflow.mass_flow_rate=mass_flow
            sim.reinitialize()
            sim.advance(time)
            #store the results
            gas_mole_fracs[:,i]=np.round(gas.X,7)
            surf_fracs[:,i]=np.round(surf.X,7)
            
        output_filename='results.txt'
        data=np.c_[rxn_time,gas_mole_fracs.T,surf_fracs.T]
        names=list(['Time'] + gas.species_names+surf.species_names)
        df=pd.DataFrame(data, columns=[names])
        df.to_csv(output_filename, index=False)

            
#Provide the temperature in °C and the pressure in bar.
methanation(350, 5)